In [2]:
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split

# Define directories
base_dir = Path('Dataset')
output_dir = Path('SplitV2')
train_dir, val_dir, test_dir = (output_dir / 'train', output_dir / 'validation', output_dir / 'test')

# Ratios
train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1

# Create necessary directories
for dir_path in [train_dir, val_dir, test_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

def copy_image(src, dst):
    """Copy file from source to destination."""
    shutil.copy2(src, dst)

for class_path in base_dir.iterdir():
    if class_path.is_dir():  # Only process directories (class folders)
        images = [img for img in class_path.iterdir() if img.is_file()]  # Filter files
        
        if len(images) < 2:  # Skip classes with very few images
            print(f"Skipping {class_path.name}: Not enough images to split.")
            continue

        # Split dataset
        train_imgs, temp_imgs = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42)

        # Create class subdirectories
        for subdir in [train_dir, val_dir, test_dir]:
            (subdir / class_path.name).mkdir(parents=True, exist_ok=True)

        # Copy files using multithreading
        with ThreadPoolExecutor() as executor:
            for img_set, target_dir in zip([train_imgs, val_imgs, test_imgs], [train_dir, val_dir, test_dir]):
                executor.map(copy_image, img_set, [target_dir / class_path.name / img.name for img in img_set])

print("Dataset successfully split into training, validation, and test sets!")


Dataset successfully split into training, validation, and test sets!


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,  # Augment dataset by flipping images
    fill_mode='nearest'  # Avoid artifacts when transforming images
)

# Validation & Test: No augmentation, just rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,  # Increase if memory allows
    class_mode='categorical',
    interpolation='bilinear'
)

validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    interpolation='bilinear'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False,  # Maintain order for evaluation
    interpolation='bilinear'
)


Found 34173 images belonging to 4 classes.
Found 4273 images belonging to 4 classes.
Found 4274 images belonging to 4 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Create the model
model = Sequential([
    # Convolutional Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Convolutional Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Convolutional Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Convolutional Block 4
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Feature Extraction
    GlobalAveragePooling2D(),  # Replaces Flatten to reduce overfitting

    # Fully Connected Layer
    Dense(128, activation='relu'),
    Dropout(0.5),  # Helps prevent overfitting

    # Output Layer
    Dense(len(train_generator.class_indices), activation='softmax')  # Multi-class classification
])

# Compile model with Adam optimizer & learning rate scheduler
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model Summary
model.summary()

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,  # Increased epochs for better learning
    callbacks=[early_stopping, reduce_lr]
)


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,748 (1.62 MB)

 Trainable params: 422,788 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 25405s 48s/step - accuracy: 0.5582 - loss: 1.0680 - val_accuracy: 0.5818 - val_loss: 1.0019 - learning_rate: 0.0010
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1772s 3s/step - accuracy: 0.6849 - loss: 0.7911 - val_accuracy: 0.6101 - val_loss: 1.0538 - learning_rate: 0.0010
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1795s 3s/step - accuracy: 0.7432 - loss: 0.6654 - val_accuracy: 0.6892 - val_loss: 0.7816 - learning_rate: 0.0010
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1940s 4s/step - accuracy: 0.7742 - loss: 0.5906 - val_accuracy: 0.7166 - val_loss: 0.8180 - learning_rate: 0.0010
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1798s 3s/step - accuracy: 0.8085 - loss: 0.5212 - val_accuracy: 0.7821 - val_loss: 0.6073 - learning_rate: 0.0010
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1792s 3s/step - accuracy: 0.8262 - loss: 0.4810 - val_accuracy: 0.7618 - val_loss: 0.7470 - learning_rate: 0.0010
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1724s 3s/step - accuracy: 0.8359 -

In [8]:
model.save("my_modelv2.h5")  # Saves as a single file


NameError: name 'model' is not defined

In [2]:
model.save("EcomistModelV2.h5")

NameError: name 'model' is not defined

In [3]:
import os
from tensorflow.keras.models import Sequential

# Define the target folder
save_dir = "models/saved"  # Change this to your desired folder
os.makedirs(save_dir, exist_ok=True)  # Create folder if it doesn't exist

# Sample model (Replace this with your actual trained model)
model = Sequential()  # Use your trained model instead

# Define the full path to save the model
model_path = os.path.join(save_dir, "EcomistModelV2.h5")

# Save the model
model.save(model_path)

print(f"Model saved at: {model_path}")


Model saved at: models/saved\EcomistModelV2.h5


In [6]:
model.save("EcomistModelV2.keras")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("Model_V2/models/saved/EcomistModelV2.h5")

loss, accuracy = model.evaluate(validation_generator)

print(f"Model accuracy: {accuracy * 100:.2f}%")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("ModelV2.keras")

loss, accuracy = model.evaluate(validation_generator)

print(f"Model accuracy: {accuracy * 100:.2f}%")


In [ ]:
import os
print(os.getcwd())  # Shows your current working directory


In [ ]:
import os

def total_files(folder_path):
    return len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

Train_healthy = "Split/train/Healthy"
Train_bacterial = "Split/train/Bacterial"
Train_Fungal = "Split/train/Fungal"
Train_Lettuce = "Split/train/Lettuce"

Test_healthy = "Split/test/Healthy"
Test_bacterial = "Split/test/Bacterial"
Test_Fungal = "Split/test/Fungal"
Test_Lettuce = "Split/test/Lettuce"

Valid_healthy = "Split/validation/Healthy"
Valid_bacterial = "Split/validation/Bacterial"
Valid_Fungal = "Split/validation/Fungal"
Valid_Lettuce = "Split/validation/Lettuce"

print("Number of healthy leaf images in training set:", total_files(Train_healthy))
print("Number of bacterial leaf images in training set:", total_files(Train_bacterial))
print("Number of fungal leaf images in training set:", total_files(Train_Fungal))
print("Number of Lettuce images in training set:", total_files(Train_Lettuce))

print("="*90)

print("Number of healthy leaf images in test set:", total_files(Test_healthy))
print("Number of bacterial leaf images in test set:", total_files(Test_bacterial))
print("Number of fungal images in test set:", total_files(Test_Fungal))
print("Number of Lettuce images in training set:", total_files(Test_Lettuce))

print("="*90)

print("Number of healthy leaf images in validation set:", total_files(Valid_healthy))
print("Number of bacterial leaf images in validation set:", total_files(Valid_bacterial))
print("Number of fungal images in validation set:", total_files(Valid_Fungal))
print("Number of Lettuce images in training set:", total_files(Valid_Lettuce))


In [ ]:
from PIL import Image
import IPython.display as display

image_path = "Split/train/Healthy/h (9).jpg"  


if os.path.exists(image_path):
    
    with open(image_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=500))
else:
    print(f"File not found: {image_path}")


In [ ]:
from PIL import Image
import IPython.display as display


image_path = "Split/test/Fungal/01_0.png" 

if os.path.exists(image_path):
   
    with open(image_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=500))
else:
    print(f"File not found: {image_path}")

In [ ]:
height = 224
width = 224
channels = 3
num_classes = 3

In [ ]:
import tensorflow as tf

def create_dataset(data, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.batch(batch_size)
    return dataset


In [ ]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)

train_data = data_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    follow_links=True 
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# I-set ang mga sukat ng imahe at batch size
image_size = (224, 224)
batch_size = 32

# Gumawa ng ImageDataGenerator para i-rescale ang pixel values ng training at validation images
train_data_gen = ImageDataGenerator(rescale=1.0 / 255.0)
valid_data_gen = ImageDataGenerator(rescale=1.0 / 255.0)

# I-load ang training images mula sa direktoryo at i-process sa generator
train_generator = train_data_gen.flow_from_directory(
    'Split/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# I-load ang validation images mula sa direktoryo at i-process sa generator
valid_generator = valid_data_gen.flow_from_directory(
    'Split/validation',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Gumawa ng Sequential model para sa convolutional neural network
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),  # First convolutional layer
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling layer para paliitin ang feature map
    Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
    MaxPooling2D(pool_size=(2, 2)),  # Second max pooling layer
    Flatten(),  # Flatten layer para i-transform ang 2D data sa 1D
    Dense(128, activation='relu'),  # Fully connected layer na may 128 units
    Dropout(0.5),  # Dropout layer para maiwasan ang overfitting
    Dense(train_generator.num_classes, activation='softmax')  # Output layer para sa classification
])

# I-compile ang model gamit ang Adam optimizer, categorical crossentropy na loss, at accuracy bilang metrics
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping callback para matigil ang training kapag walang improvement sa validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Itigil ang training pagkatapos ng 5 epochs na walang improvement
    verbose=1,
    restore_best_weights=True  # Ibalik ang best weights kapag natapos ang training
)

# I-train ang model gamit ang train_generator at i-validate gamit ang valid_generator
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=valid_generator,
    callbacks=[early_stopping]  # I-apply ang early stopping sa training
)


In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns


sns.set_theme()
sns.set_context("poster")


figure(figsize=(25, 25), dpi=100)

# Plot training and validation accuracy
plt.subplot(2, 1, 1)  
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

# Plot training and validation loss
plt.subplot(2, 1, 2)  
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')


plt.tight_layout()
plt.show()


In [ ]:
model.save("Model.h5")

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model_path = "EcomistModelV2.h5"  # Adjust the path if needed

try:
    model = load_model(model_path)
    print("Loaded a full model (architecture + weights).")
    model.summary()  # Print model architecture
except:
    print("The file contains only weights. You need the original model architecture to load them.")



The file contains only weights. You need the original model architecture to load them.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input

def build_model():
    input_layer = Input(shape=(224, 224, 3))

    # Convolutional Block 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Convolutional Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Convolutional Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Convolutional Block 4
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Feature Extraction
    x = GlobalAveragePooling2D()(x)

    # Fully Connected Layer
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output Layer (Update based on class count)
    output_layer = Dense(4, activation='softmax')(x)  # Update with correct number of classes

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Rebuild the model
model = build_model()
model.summary()

# Load the weights
model.load_weights("EcomistModelV2.h5")

print("✅ Model rebuilt & weights loaded successfully!")





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 224, 224, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 64)      256

In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a test image (replace with an actual image path)
img_path = r"SplitV2\test\Healthy\-_jpg.rf.a2935bd9f9dc97942b14b598c4c16012.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

# Make a prediction
predictions = model.predict(img_array)
print("Predictions:", predictions)
print("Predicted class:", np.argmax(predictions))


1/1 [==============================] - 0s 45ms/step
Predictions: [[1.2593208e-03 2.7327300e-05 9.9869269e-01 2.0632433e-05]]
Predicted class: 2


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths (make sure these are correct)
train_dir = "SplitV2/test"

# Recreate ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)  # Ensure the same preprocessing as before

# Load training data again
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Now get class labels
class_labels = list(train_generator.class_indices.keys())

print("Class Index Mapping:", train_generator.class_indices)


Found 4274 images belonging to 4 classes.
Class Index Mapping: {'Bacterial': 0, 'Fungal': 1, 'Healthy': 2, 'Lettuce': 3}


In [13]:
# Save as TensorFlow SavedModel format
model.save("LettuceModelV2")

# Load the saved model when needed
loaded_model = tf.keras.models.load_model("LettuceModelV2")

print("✅ Successfully loaded from SavedModel format!")


INFO:tensorflow:Assets written to: LettuceModelV2\assets


INFO:tensorflow:Assets written to: LettuceModelV2\assets


✅ Successfully loaded from SavedModel format!
